# 알고리즘 짜야합니다

# 라이브러리 및 모델 불러오기

In [72]:
from flask import Flask ,render_template
from flask import request, redirect
from konlpy.tag import Komoran # 형태소 분석 라이브러리
from konlpy.tag import Twitter # 형태소 분석 라이브러리
from konlpy.tag import Kkma,Okt
from moviepy.editor import * # 영상을 오디오 파일로 변환
from moviepy.editor import VideoFileClip, concatenate_videoclips
import moviepy.editor as mp
from pytube import YouTube # 유튜브 영상 다운로드 또는 불러오기
import pytube
import tqdm as tq

In [73]:
import kss # 텍스트 문장으로 바꾸는 라이브러리
import speech_recognition as sr # 오디오 파일 또는 음성을 텍스트로 변환
import pandas as pd
# BOW = BAG of WORD : 단어가방, 단어모음, 단어사전
from sklearn.feature_extraction.text import CountVectorizer
# 위 도구는 빈도수 기반 벡터화 도구

# 알고리즘 시작

# 형태소 구분하는 함수
- 사용해야할 품사가 생각보다 많음 ( komoran 기준 ) 
  - 명사 NN -> 일반명사 NNG // 고유명사 NNP // 의존명사 NNB

In [70]:
def lemmatize(word):
    morphtags = Komoran().pos(word)
    if morphtags[0][1] == 'NNG' or morphtags[0][1] == 'NNP':
        return morphtags[0][0]
    


In [55]:
word = '앞서 전화 드린 것처럼 지금부터 이른바 청와대 비서실 새로 지목된 최순실 관련 소식을 집중보도 하겠습니다 지난주 JTBC는 최순실 씨 최측근이라고 하는 고영태 씨를 주 대한 내용을 단독으로 보내 드렸습니다 최순실 씨가 유일하게 잘하는 것이 대통령 연설문 수정하는 것이다라는 내용이었는데요이 내용을 보도하자 청와대 이원종 비서실장은 정상적인 사람이면 믿을 수 있겠느냐 공부할 때도 있을 수 없는 얘기 다 이렇게 얘기 한 바 있습니다 JTBC가 몇 시에 말을 보도한 배경에는 사실 또 다른 믿기 어려운 정황이 있기 때문에 왔습니다 JTBC 취재팀은 최순실 씨 컴퓨터 입수해서 분석을 했습니다 태식아 대통령 연설문 에바다 봤다는 사실을 확인할 수 있었습니다 그런데 3시가 연설문 44개를 파일형태로 받은 실점을 너무도 대통령이 연설을 하기 의견이었습니다 먼저 김필중 기자의 단독 벌 벌 벌 이어가겠습니다 제수씨 사무실에 있던 pc에 저장된 파일들입니다 각종 문서도 가득합니다 파일은 모두 200여개의 일입니다 그런데 씨가 보관 중인 파일에 대부분이 청와대 와 관련된 내용이었습니다 기회 되면 특히 제시가 대통령 연설문 수정했다는 최초의 책은 고영태의 준수과 관련해 연설문의 주목했습니다 첼시가 갖고 있던 연설문 또는 공식 빠른한테 파일엔 모드 44개 없습니다 대선후보 시절에 박 대통령의 유선문의 비롯해 대통령 취임 연설문 드리기로 했습니다 그런데 최씨가 이문권 에바다 여러분 시장은 대통령이 실제 발언했던 것보다 길게는 4월이나 없었습니다 상당수 대통령 연설문 사전에 청와대 내부에서도 도움이 되지 않는다는 점을 감안하면 연설문이 사전에 청와대 아무거나 최씨에게 전달되었던 사실은 이른바 비선실세 놀란 거 관련해서 큰 날 것으로 보입니다 JTBC 김필 주입니다'

word_list = kss.split_sentences(word)

from collections import Counter

okja = []

for line in word_list:
    okja.append(line)
print(okja)

print('------------------------------------------------------------')

twitter = Twitter()
sentences_tag = []
for i in okja:
    morph= twitter.pos(i)
    sentences_tag.append(morph)
    
print(sentences_tag)


[Korean Sentence Splitter]: Initializing Pynori...
[Korean Sentence Splitter]: C:\Users\smhrd\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')



['앞서 전화 드린 것처럼 지금부터 이른바 청와대 비서실 새로 지목된 최순실 관련 소식을 집중보도 하겠습니다', '지난주 JTBC는 최순실 씨 최측근이라고 하는 고영태 씨를 주 대한 내용을 단독으로 보내 드렸습니다', '최순실 씨가 유일하게 잘하는 것이 대통령 연설문 수정하는 것이다라는 내용이었는데요', '이 내용을 보도하자 청와대 이원종 비서실장은 정상적인 사람이면 믿을 수 있겠느냐 공부할 때도 있을 수 없는 얘기 다 이렇게 얘기 한 바 있습니다', 'JTBC가 몇 시에 말을 보도한 배경에는 사실 또 다른 믿기 어려운 정황이 있기 때문에 왔습니다', 'JTBC 취재팀은 최순실 씨 컴퓨터 입수해서 분석을 했습니다', '태식아 대통령 연설문 에바다 봤다는 사실을 확인할 수 있었습니다', '그런데 3시가 연설문 44개를 파일형태로 받은 실점을 너무도 대통령이 연설을 하기 의견이었습니다', '먼저 김필중 기자의 단독 벌 벌 벌 이어가겠습니다', '제수씨 사무실에 있던 pc에 저장된 파일들입니다', '각종 문서도 가득합니다', '파일은 모두 200여개의 일입니다', '그런데 씨가 보관 중인 파일에 대부분이 청와대 와 관련된 내용이었습니다', '기회 되면 특히 제시가 대통령 연설문 수정했다는 최초의 책은 고영태의 준수과 관련해 연설문의 주목했습니다', '첼시가 갖고 있던 연설문 또는 공식 빠른한테 파일엔 모드 44개 없습니다', '대선후보 시절에 박 대통령의 유선문의 비롯해 대통령 취임 연설문 드리기로 했습니다', '그런데 최씨가 이문권 에바다 여러분 시장은 대통령이 실제 발언했던 것보다 길게는 4월이나 없었습니다', '상당수 대통령 연설문 사전에 청와대 내부에서도 도움이 되지 않는다는 점을 감안하면 연설문이 사전에 청와대 아무거나 최씨에게 전달되었던 사실은 이른바 비선실세 놀란 거 관련해서 큰 날 것으로 보입니다 JTBC 김필 주입니다']
------------------------------------------------------------
[[('앞서', 'N

In [56]:
noun_adj_list=[]
for i1 in sentences_tag:
    for word, tag in i1:
        if tag in ['Noun','Verb','Number','Adjective','Adverb']:
            noun_adj_list.append(word)
print(noun_adj_list)

['앞서', '전화', '드린', '것', '지금', '이른바', '청와대', '비서실', '새로', '지목', '된', '최', '실', '관련', '소식', '집중', '보도', '하겠습니다', '지난주', '는', '최', '실', '씨', '최측근', '하는', '영태', '씨', '주', '대한', '내용', '단독', '보내', '드렸습니다', '최', '실', '씨', '유일하게', '하는', '것', '대통령', '설문', '수정', '하는', '것', '내용', '이었는데요', '이', '내용', '보도', '하자', '청와대', '이원종', '비서실', '정상', '사람', '이면', '믿을', '수', '있겠느냐', '공부', '할', '때', '있을', '수', '없는', '얘기', '다', '이렇게', '얘기', '한', '바', '있습니다', '가', '몇', '시', '말', '보도', '배경', '사실', '또', '다른', '믿기', '어려운', '정황', '있기', '때문', '왔습니다', '취재', '팀', '최', '실', '씨', '컴퓨터', '입수', '해서', '분석', '했습니다', '태식', '대통령', '설문', '에바', '봤다는', '사실', '확인', '할', '수', '있었습니다', '3시', '설문', '44', '개', '파일', '형태', '받은', '실점', '너무도', '대통령', '연설', '하기', '의견', '이었습니다', '먼저', '김필중', '기자', '단독', '벌', '벌', '벌', '이어가겠습니다', '제수', '사무실', '있던', '저장', '된', '파일', '들입니다', '각종', '문서', '가득합니다', '파일', '모두', '200', '여개', '일', '입니다', '씨', '보관', '중인', '파일', '대부분', '청와대', '와', '관련', '된', '내용', '이었습니다', '기회', '되면', '특히', '제시', '대통령', '설문', '수정', '했다

In [57]:
for i in range(len(noun_adj_list)):
    print(lemmatize(noun_adj_list[i]))
    if lemmatize(noun_adj_list[i]) != None :
        noun_adj_list[i] = lemmatize(noun_adj_list[i])
        print(noun_adj_list)
        
arr_list = noun_adj_list
print(arr_list)

 # 영상합치기 부분으로 넘어가기

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [69]:
for i in range(len(noun_adj_list)):
    morphtags = Komoran().pos(noun_adj_list[i])
    print(noun_adj_list[i])
    print('---------')
    print(morphtags)
    print('=========')
    print(lemmatize(noun_adj_list[i])) # NNG, NNP

앞서
---------
[('앞서', 'MAG')]
None
전화
---------
[('전화', 'NNG')]
전화다
드린
---------
[('드리', 'VV'), ('ㄴ', 'ETM')]
None
것
---------
[('것', 'NNB')]
None
지금
---------
[('지금', 'MAG')]
None
이른바
---------
[('이른바', 'MAJ')]
None
청와대
---------
[('청와대', 'NNP')]
청와대다
비서실
---------
[('비서실', 'NNP')]
비서실다
새로
---------
[('새로', 'MAG')]
None
지목
---------
[('지목', 'NNG')]
지목다
된
---------
[('되', 'VV'), ('ㄴ', 'ETM')]
None
최
---------
[('최', 'NNP')]
최다
실
---------
[('실', 'NNG')]
실다
관련
---------
[('관련', 'NNG')]
관련다
소식
---------
[('소식', 'NNG')]
소식다
집중
---------
[('집중', 'NNG')]
집중다
보도
---------
[('보도', 'NNG')]
보도다
하겠습니다
---------
[('하', 'VV'), ('겠', 'EP'), ('습니다', 'EC')]
None
지난주
---------
[('지난주', 'NNG')]
지난주다
는
---------
[('늘', 'VV'), ('ㄴ', 'ETM')]
None
최
---------
[('최', 'NNP')]
최다
실
---------
[('실', 'NNG')]
실다
씨
---------
[('씨', 'NNB')]
None
최측근
---------
[('최', 'XPN'), ('측근', 'NNG')]
None
하는
---------
[('하', 'VV'), ('는', 'ETM')]
None
영태
---------
[('영태', 'NNP')]
영태다
씨
---------
[('씨', 'NNB')]
None
주
---------


None
설문
---------
[('설문', 'NNP')]
설문다
주목
---------
[('주목', 'NNG')]
주목다
했습니다
---------
[('하', 'VV'), ('았', 'EP'), ('습니다', 'EC')]
None
첼시
---------
[('첼시', 'NNP')]
첼시다
갖고
---------
[('갖', 'VV'), ('고', 'EC')]
None
있던
---------
[('있', 'VV'), ('던', 'ETM')]
None
설문
---------
[('설문', 'NNP')]
설문다
또는
---------
[('또는', 'MAJ')]
None
공식
---------
[('공식', 'NNG')]
공식다
빠른
---------
[('빠르', 'VA'), ('ㄴ', 'ETM')]
None
파일
---------
[('파일', 'NNP')]
파일다
모드
---------
[('모드', 'NNP')]
모드다
44
---------
[('44', 'SN')]
None
개
---------
[('개', 'NNB')]
None
없습니다
---------
[('없', 'VA'), ('습니다', 'EC')]
None
대선
---------
[('대선', 'NNG')]
대선다
후보
---------
[('후보', 'NNG')]
후보다
시절
---------
[('시절', 'NNG')]
시절다
박
---------
[('박', 'NNP')]
박다
대통령
---------
[('대통령', 'NNG')]
대통령다
유선
---------
[('유선', 'NNP')]
유선다
문의
---------
[('문', 'NNG'), ('의', 'JKG')]
문다
비롯
---------
[('비롯', 'XR')]
None
해
---------
[('하', 'VV'), ('아', 'EC')]
None
대통령
---------
[('대통령', 'NNG')]
대통령다
취임
---------
[('취임', 'NNG')]
취임다
설문
---------
[('설문', 'NNP')